<a href="https://colab.research.google.com/github/shahriariit/Violence_Videos_Detection/blob/main/Violence_Detection_from_Movies_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python
!pip install tensorflow
!pip install moviepy

In [2]:
from google.colab import drive
import cv2
import os
import math
import matplotlib.pyplot as plt
import numpy as np
import csv
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score, cohen_kappa_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef
from imblearn.metrics import geometric_mean_score
from imblearn.metrics import sensitivity_score
from imblearn.metrics import specificity_score

In [3]:
drive.mount('/content/drive')
dataset_path = '/content/drive/MyDrive/MDataset'

Mounted at /content/drive


In [4]:
def load_images_from_folder(folder, label):
    """Load images from a folder, preprocess, and assign labels."""
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        # Load and preprocess image
        img = load_img(img_path, target_size=(224, 224))  # Resize image to 224x224
        img = img_to_array(img) / 255.0  # Normalize to [0, 1]
        images.append(img)
        labels.append(label)
    return np.array(images), np.array(labels)


def build_transfer_learning_model(base_model_fn, input_shape=(224, 224, 3), learning_rate=0.001):
    """
    Builds a binary classification model using a specified base model function.

    Parameters:
    - base_model_fn: a function from keras.applications (e.g., VGG16, ResNet50V2)
    - input_shape: input shape of the images
    - learning_rate: learning rate for the optimizer

    Returns:
    - Compiled Keras Model
    """
    # Load the base model without the top classification layer
    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)

    # Final model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile model
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

def evaluate(X_test, y_test, output_file="classification_metrics.csv"):
    # Predict probabilities

    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)

        # Write the header row
        writer.writerow(["Accuracy", "Cohen-Kappa", "Precision", "Recall", "F1-Score", "AUROC", "MCC", "Specificity", "G-Mean", "Type I Error","Type II Error"])

        y_probs = model_vgg16.predict(X_test)
        y_pred = (y_probs > 0.5).astype(int).flatten()

        # Classification Report (includes precision, recall, F1-score)
        print("Classification Report:\n", classification_report(y_test, y_pred))

        # Individual Metrics
        accuracy = accuracy_score(y_test, y_pred)
        cohen_kappa = cohen_kappa_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        auroc = roc_auc_score(y_test, y_probs, average='weighted')
        mcc = matthews_corrcoef(y_test, y_pred)
        specificity = specificity_score(y_test, y_pred, average='weighted')
        g_mean = geometric_mean_score(y_test, y_pred, average='weighted')
        type_i_error = 1 - g_mean
        type_ii_error = 1 - specificity


        writer.writerow([accuracy, cohen_kappa, precision, recall, f1, auroc, mcc, specificity, g_mean, type_i_error, type_ii_error])

        # Display results
        print("Accuracy:", accuracy)
        print("Cohen's Kappa:", cohen_kappa)
        print("Precision:", precision)
        print("Recall:", recall)
        print("F1-Score:", f1)
        print("AUROC:", auroc)
        print("Matthews Correlation Coefficient (MCC):", mcc)
        print("Specificity",specificity)
        print("Geometric Mean",g_mean)
        print("Type I Error",type_i_error)
        print("Type II Error",type_ii_error)

In [5]:
# Paths to the frames
nonviolence_folder = '/content/drive/MyDrive/MDataset/noFights'
violence_folder = '/content/drive/MyDrive/MDataset/fights'

In [6]:
# Load images and labels for each class
violence_images, violence_labels = load_images_from_folder(violence_folder, 1)  # Label 1 for Violence
nonviolence_images, nonviolence_labels = load_images_from_folder(nonviolence_folder, 0)  # Label 0 for Non-Violence

# Combine the datasets
X = np.concatenate((violence_images, nonviolence_images), axis=0)
y = np.concatenate((violence_labels, nonviolence_labels), axis=0)

# Shuffle the dataset
X, y = shuffle(X, y, random_state=42)

print(f"Dataset loaded and shuffled.")
print(f"Total samples: {len(y)}")
print(f"Shape of X: {X.shape}, Shape of y: {y.shape}")

Dataset loaded and shuffled.
Total samples: 428
Shape of X: (428, 224, 224, 3), Shape of y: (428,)


In [7]:
# Split dataset into training and testin
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
model_vgg16 = build_transfer_learning_model(VGG16)
model_vgg16.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_vgg16 = model_vgg16.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_vgg16}")
evaluate(X_test, y_test,"vgg16_classification.csv")

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.7347 - loss: 0.5524 - val_accuracy: 0.8837 - val_loss: 0.3361
Epoch 2/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 18s 196ms/step - accuracy: 0.9316 - loss: 0.2647 - val_accuracy: 0.9535 - val_loss: 0.1636
Epoch 3/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 215ms/step - accuracy: 0.9324 - loss: 0.1851 - val_accuracy: 0.9651 - val_loss: 0.1234
Epoch 4/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 217ms/step - accuracy: 0.9516 - loss: 0.1238 - val_accuracy: 0.9884 - val_loss: 0.0843
Epoch 5/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 198ms/step - accuracy: 0.9716 - loss: 0.0814 - val_accuracy: 0.9884 - val_loss: 0.0603
Epoch 6/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 197ms/step - accuracy: 0.9843 - loss: 0.0794 - val_accuracy: 0.9884 - val_loss: 0.0501
Accuracy: 0.9883720874786377
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 411ms/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.97      0.

In [13]:
model_vgg19 = build_transfer_learning_model(VGG19)
model_vgg19.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_vgg19 = model_vgg19.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_vgg19}")
evaluate(X_test, y_test,"vgg19_classification.csv")

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 678ms/step - accuracy: 0.6574 - loss: 0.6113 - val_accuracy: 0.9419 - val_loss: 0.3367
Epoch 2/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 265ms/step - accuracy: 0.8567 - loss: 0.3648 - val_accuracy: 0.9419 - val_loss: 0.1949
Epoch 3/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 261ms/step - accuracy: 0.9245 - loss: 0.2234 - val_accuracy: 0.9302 - val_loss: 0.1584
Epoch 4/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 260ms/step - accuracy: 0.9585 - loss: 0.1657 - val_accuracy: 0.9884 - val_loss: 0.1057
Epoch 5/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 263ms/step - accuracy: 0.9472 - loss: 0.1290 - val_accuracy: 1.0000 - val_loss: 0.0832
Epoch 6/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 263ms/step - accuracy: 0.9636 - loss: 0.1111 - val_accuracy: 1.0000 - val_loss: 0.0654
Accuracy: 1.0
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.97      0.99        35


In [12]:
model_mobilenetv3 = build_transfer_learning_model(MobileNetV3Large)
model_mobilenetv3.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_mobilenetv3 = model_mobilenetv3.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_mobilenetv3}")
evaluate(X_test, y_test,"mobilenetv3_classification.csv")

12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.5890 - loss: 0.6762 - val_accuracy: 0.8372 - val_loss: 0.4439
Epoch 2/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 16s 46ms/step - accuracy: 0.8260 - loss: 0.4602 - val_accuracy: 0.9419 - val_loss: 0.3062
Epoch 3/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.9091 - loss: 0.3429 - val_accuracy: 0.9767 - val_loss: 0.2375
Epoch 4/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.9256 - loss: 0.2707 - val_accuracy: 0.9767 - val_loss: 0.1883
Epoch 5/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.9587 - loss: 0.1973 - val_accuracy: 0.9767 - val_loss: 0.1372
Epoch 6/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.9492 - loss: 0.1816 - val_accuracy: 0.9767 - val_loss: 0.1222
Accuracy: 0.9767441749572754
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.97      0.99   

In [9]:
model_resnet50v2 = build_transfer_learning_model(ResNet50V2)
model_resnet50v2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_resnet50v2 = model_resnet50v2.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_resnet50v2}")
evaluate(X_test, y_test,"resnet50v2_classification.csv")

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.8201 - loss: 0.3725 - val_accuracy: 0.9884 - val_loss: 0.0391
Epoch 2/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 15s 106ms/step - accuracy: 0.9967 - loss: 0.0072 - val_accuracy: 0.9535 - val_loss: 0.0959
Epoch 3/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - accuracy: 1.0000 - loss: 0.0040 - val_accuracy: 0.9884 - val_loss: 0.0377
Epoch 4/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - accuracy: 1.0000 - loss: 0.0024 - val_accuracy: 0.9884 - val_loss: 0.0173
Epoch 5/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - accuracy: 0.9983 - loss: 0.0046 - val_accuracy: 1.0000 - val_loss: 0.0069
Epoch 6/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - accuracy: 1.0000 - loss: 0.0025 - val_accuracy: 0.9884 - val_loss: 0.0132
Accuracy: 0.9883720874786377
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 171ms/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.97      0.9

In [11]:
model_inceptionresnetv2 = build_transfer_learning_model(InceptionResNetV2)
model_inceptionresnetv2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_inceptionresnetv2 = model_inceptionresnetv2.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_inceptionresnetv2}")
evaluate(X_test, y_test,"inceptionresnetv2_classification.csv")

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 77s 4s/step - accuracy: 0.6470 - loss: 0.9478 - val_accuracy: 0.8372 - val_loss: 0.7678
Epoch 2/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 29s 214ms/step - accuracy: 0.8919 - loss: 0.3227 - val_accuracy: 0.9535 - val_loss: 0.1288
Epoch 3/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 189ms/step - accuracy: 0.9506 - loss: 0.1484 - val_accuracy: 0.9884 - val_loss: 0.0709
Epoch 4/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 185ms/step - accuracy: 0.9601 - loss: 0.0941 - val_accuracy: 0.9651 - val_loss: 0.0529
Epoch 5/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 209ms/step - accuracy: 0.9691 - loss: 0.0734 - val_accuracy: 0.9884 - val_loss: 0.0387
Epoch 6/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 186ms/step - accuracy: 0.9778 - loss: 0.0308 - val_accuracy: 0.9884 - val_loss: 0.0303
Accuracy: 0.9883720874786377
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.97      

In [10]:
model_densenet121 = build_transfer_learning_model(DenseNet121)
model_densenet121.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_densenet121 = model_densenet121.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_densenet121}")
evaluate(X_test, y_test,"densenet121_classification.csv")

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 74s 4s/step - accuracy: 0.5148 - loss: 1.0598 - val_accuracy: 0.8372 - val_loss: 0.4584
Epoch 2/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 25s 111ms/step - accuracy: 0.8785 - loss: 0.3762 - val_accuracy: 0.9535 - val_loss: 0.0931
Epoch 3/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 0.9554 - loss: 0.1035 - val_accuracy: 0.9884 - val_loss: 0.0528
Epoch 4/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - accuracy: 0.9788 - loss: 0.0620 - val_accuracy: 0.9884 - val_loss: 0.0391
Epoch 5/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 0.9976 - loss: 0.0188 - val_accuracy: 0.9884 - val_loss: 0.0320
Epoch 6/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - accuracy: 1.0000 - loss: 0.0177 - val_accuracy: 0.9884 - val_loss: 0.0274
Accuracy: 0.9883720874786377
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.97      0.

In [16]:
model_densenet169 = build_transfer_learning_model(DenseNet169)
model_densenet169.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_densenet169 = model_densenet169.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_densenet169}")
evaluate(X_test, y_test,"densenet169_classification.csv")

51877672/51877672 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 80s 4s/step - accuracy: 0.7476 - loss: 0.5673 - val_accuracy: 0.9419 - val_loss: 0.1178
Epoch 2/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 25s 136ms/step - accuracy: 0.9365 - loss: 0.1228 - val_accuracy: 0.9884 - val_loss: 0.0383
Epoch 3/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step - accuracy: 0.9942 - loss: 0.0163 - val_accuracy: 0.9884 - val_loss: 0.0236
Epoch 4/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - accuracy: 0.9990 - loss: 0.0124 - val_accuracy: 1.0000 - val_loss: 0.0108
Epoch 5/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - accuracy: 0.9967 - loss: 0.0097 - val_accuracy: 1.0000 - val_loss: 0.0069
Epoch 6/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step - accuracy: 1.0000 - loss: 0.0039 - val_accuracy: 1.0000 - val_loss: 0.0075
Accuracy: 1.0
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.97      0.99        35
  

In [14]:
model_densenet201 = build_transfer_learning_model(DenseNet201)
model_densenet201.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_densenet201 = model_densenet201.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_densenet201}")
evaluate(X_test, y_test,"densenet201_classification.csv")

74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 103s 6s/step - accuracy: 0.6919 - loss: 0.5320 - val_accuracy: 1.0000 - val_loss: 0.0199
Epoch 2/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 152ms/step - accuracy: 0.9828 - loss: 0.0527 - val_accuracy: 1.0000 - val_loss: 0.0085
Epoch 3/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 148ms/step - accuracy: 0.9986 - loss: 0.0107 - val_accuracy: 1.0000 - val_loss: 0.0030
Epoch 4/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 151ms/step - accuracy: 0.9958 - loss: 0.0138 - val_accuracy: 1.0000 - val_loss: 0.0043
Epoch 5/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 179ms/step - accuracy: 1.0000 - loss: 0.0058 - val_accuracy: 1.0000 - val_loss: 0.0014
Epoch 6/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 150ms/step - accuracy: 0.9967 - loss: 0.0068 - val_accuracy: 1.0000 - val_loss: 0.0013
Accuracy: 1.0
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.97      0.99        35
  

In [15]:
model_inceptionv3 = build_transfer_learning_model(InceptionV3)
model_inceptionv3.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_inceptionv3 = model_inceptionv3.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_inceptionv3}")
evaluate(X_test, y_test,"inceptionv3_classification.csv")

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.5999 - loss: 1.6354 - val_accuracy: 0.6977 - val_loss: 0.6163
Epoch 2/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 89ms/step - accuracy: 0.8493 - loss: 0.3686 - val_accuracy: 0.9535 - val_loss: 0.1390
Epoch 3/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - accuracy: 0.9294 - loss: 0.1653 - val_accuracy: 0.9767 - val_loss: 0.0594
Epoch 4/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - accuracy: 0.9697 - loss: 0.0657 - val_accuracy: 0.9884 - val_loss: 0.0434
Epoch 5/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - accuracy: 0.9852 - loss: 0.0403 - val_accuracy: 0.9884 - val_loss: 0.0274
Epoch 6/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - accuracy: 0.9919 - loss: 0.0217 - val_accuracy: 0.9884 - val_loss: 0.0257
Accuracy: 0.9883720874786377
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.97      0.99   